In [5]:
import orjson

In [14]:
with open('eval_results/gorilla-openfunctions-v2-5.0bpw-h6-exl2-pretrained_score.json', 'rb') as f:
    results = orjson.loads(f.read())
with open('eval_outputs/gorilla-openfunctions-v2-5.0bpw-h6-exl2-pretrained.json', 'rb') as f:
    infer_output = orjson.loads(f.read())
with open('../../data/nlp.jsonl', 'rb') as f:
    label = [orjson.loads(line.strip()) for line in f if line.strip() != ""]

In [15]:
failed = {k: v for k, v in results.items() if v['target'] < 1 or v['tool'] < 1}
failed_heading = {k: v for k, v in results.items() if v['heading'] < 1}

In [8]:
# 230 Turret Foxtrot, engage the target at heading two seven five, red and yellow missile, deploy surface-to-air missiles. Turret Foxtrot, prepare for strike. It give [{'heading': '275', 'tool': 'red and yellow missile', 'target': 'deploy surface-to-air missiles'}]

In [17]:
failed_heading

{'359': {'heading': 0.0, 'target': 0, 'tool': 0}}